In [ ]:
#ARQUIVO PARA GERAR O CSV

import pandas as pd
from datetime import datetime, timedelta

print("Iniciando a geração do arquivo COMPLETO de predições para o Power BI...")

try:
    df = pd.read_csv('df_t_cidades.csv', sep=',')
    df['date_purchase'] = pd.to_datetime(df['date_purchase'])
    df = df.sort_values(by=['fk_contact', 'date_purchase'])
    print("Dados carregados e preparados.")
except FileNotFoundError:
    print("ERRO: Arquivo 'df_t_cidades.csv' não foi encontrado.")
    exit()


print("Calculando todas as métricas por cliente...")


grouped = df.groupby('fk_contact')


df_predicao = grouped.agg(
    ultima_compra_registrada=('date_purchase', 'max'),
    total_compras_historico=('date_purchase', 'size'),
    proximo_destino_provavel=('destination_city_departure', lambda x: x.mode().iloc[0] if not x.mode().empty else None)
)


diferencas_por_grupo = grouped['date_purchase'].diff()

frequencia_media = diferencas_por_grupo.groupby(df['fk_contact']).mean()

df_predicao['frequencia_media_compra_dias'] = frequencia_media.dt.days.round(0)

print("Projetando datas para o futuro...")
hoje = pd.to_datetime(datetime.now())

def projetar_data(row):
    freq_dias = row['frequencia_media_compra_dias']
    if pd.isna(freq_dias):
        return None

    if freq_dias == 0:
        return row['ultima_compra_registrada'] + timedelta(days=1)

    proxima_data = row['ultima_compra_registrada'] + timedelta(days=freq_dias)

    while proxima_data < hoje:
        proxima_data += timedelta(days=freq_dias)

    return proxima_data

df_predicao['proxima_data_compra_provavel'] = df_predicao.apply(projetar_data, axis=1)

print("Calculando previsão de viagem para os próximos 30 dias...")
limite_30_dias = hoje + timedelta(days=30)

def verificar_viagem_proxima(data_prevista):
    if pd.isna(data_prevista):
        return 'Não'
    if hoje <= data_prevista <= limite_30_dias:
        return 'Sim'
    else:
        return 'Não'

df_predicao['viaja_em_30_dias'] = df_predicao['proxima_data_compra_provavel'].apply(verificar_viagem_proxima)

print("Formatando e salvando o arquivo final...")

df_predicao_final = df_predicao.reset_index()
df_predicao_final['ultima_compra_registrada'] = pd.to_datetime(df_predicao_final['ultima_compra_registrada']).dt.date
df_predicao_final['proxima_data_compra_provavel'] = pd.to_datetime(df_predicao_final['proxima_data_compra_provavel']).dt.date

try:
    df_predicao_final.to_csv('predicoes_clientes_completo.csv', index=False, encoding='utf-8')
    print("\nSUCESSO! Arquivo 'predicoes_clientes_completo.csv' foi criado com todas as colunas solicitadas.")
except Exception as e:
    print(f"\nERRO ao salvar o arquivo: {e}")